# Анализ лояльности пользователей Яндекс Афиши

- Автор: Литвинов Олег Игоревич
- Дата: 

### Цели и задачи проекта



### Этапы выполнения проекта

#### Подготовка к работе
1. Создание локального репозитория (venv, .gitignore)
2. Создание репозитория на Github
3. Создание .env с конфигурацией данных для подключения к бд
#### Загрузка данных и знакомство с ними
1. Подключение к бд через .env
1. SQL запрос для выгрузки в датафрейм
2. Общая информация о данных
#### Предобработка данных
1. Преобразование к необходимому типу
2. Анализ пропусков
3. Обработка пропущенных значений
4. Проверка на явные и неявные дубликаты
#### Создание профиля пользователя
1. Агрегированные признаки, описывающие поведение и профиль каждого пользователя
2. Анализ данных в разрезе каждого пользователя
#### Исследовательский анализ данных
1. Исследование признаков первого заказа и их связи с возвращением на платформу
2. Исследование вернувшихся пользователей
4. Исследование поведения пользователей через показатели выручки и состава заказа
5. Исследование временных характеристик первого заказа и их влияния на повторные покупки
6. Корреляционный анализ количества покупок и признаков пользователя
#### Общие выводы и рекомендации
#### Финализация проекта и публикация в Git
1. Создание requirements.txt
2. Финальный коммит и пуш в Github 

## Подготовка к работе

Все шаги выполнены локально. Вирт окружение создано, локальный репозиторий создан.

## Загрузка данных и знакомство с ними

In [1]:
# загружаем необходимые библиотеки

import pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

In [2]:
# загружаем переменные из .env

load_dotenv()

db_config = {
    'user': os.getenv('DB_USER'),
    'pwd': os.getenv('DB_PASSWORD'), 
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'db': os.getenv('DB_NAME')
}

In [3]:
# создаём строку для соединения с базой данных из переменных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [4]:
# подключаемся

engine = create_engine(connection_string)

In [5]:
# пишем запрос в переменную

query = '''
WITH set_config_precode AS (
  SELECT set_config('synchronize_seqscans', 'off', true)
)
SELECT  p.user_id,
        p.device_type_canonical,
        p.order_id,
        p.created_dt_msk AS order_dt,
        p.created_ts_msk AS order_ts,
        p.currency_code,
        p.revenue,
        p.tickets_count,
        EXTRACT(DAY FROM (created_dt_msk - LAG(created_dt_msk) OVER(
            PARTITION BY user_id 
            ORDER BY created_dt_msk
          ))) AS days_since_prev,
        p.event_id,
        e.event_name_code AS event_name,
        e.event_type_main,
        p.service_name,
        r.region_name,
        c.city_name
FROM afisha.purchases AS p
JOIN afisha.events e USING(event_id)
LEFT JOIN afisha.city c USING(city_id)
LEFT JOIN afisha.regions r USING(region_id)
WHERE p.device_type_canonical IN ('mobile', 'desktop') and e.event_type_main != 'фильм'
ORDER BY p.user_id
;
'''

In [6]:
# получаем результат, записываем в датафрейм

df = pd.read_sql_query(query, con=engine) 
df.head()

,user_id,device_type_canonical,order_id,order_dt,order_ts,currency_code,revenue,tickets_count,days_since_prev,event_id,event_name,event_type_main,service_name,region_name,city_name
0,0002849b70a3ce2,mobile,4359165,2024-08-20,2024-08-20 16:08:03,rub,1521.94,4,NaN,169230,f0f7b271-04eb-4af6-bcb8-8f05cf46d6ad,театр,Край билетов,Каменевский регион,Глиногорск
1,0005ca5e93f2cf4,mobile,7965605,2024-07-23,2024-07-23 18:36:24,rub,289.45,2,NaN,237325,40efeb04-81b7-4135-b41f-708ff00cc64c,выставки,Мой билет,Каменевский регион,Глиногорск
2,0005ca5e93f2cf4,mobile,7292370,2024-10-06,2024-10-06 13:56:02,rub,1258.57,4,75.0,578454,01f3fb7b-ed07-4f94-b1d3-9a2e1ee5a8ca,другое,За билетом!,Каменевский регион,Глиногорск
3,000898990054619,mobile,1139875,2024-07-13,2024-07-13 19:40:48,rub,8.49,2,NaN,387271,2f638715-8844-466c-b43f-378a627c419f,другое,Лови билет!,Североярская область,Озёрск
4,000898990054619,mobile,972400,2024-10-04,2024-10-04 22:33:15,rub,1390.41,3,83.0,509453,10d805d3-9809-4d8a-834e-225b7d03f95d,стендап,Билеты без проблем,Озернинский край,Родниковецк


In [7]:
# общая информация о датасете

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290611 entries, 0 to 290610
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   user_id                290611 non-null  object        
 1   device_type_canonical  290611 non-null  object        
 2   order_id               290611 non-null  int64         
 3   order_dt               290611 non-null  datetime64[ns]
 4   order_ts               290611 non-null  datetime64[ns]
 5   currency_code          290611 non-null  object        
 6   revenue                290611 non-null  float64       
 7   tickets_count          290611 non-null  int64         
 8   days_since_prev        268678 non-null  float64       
 9   event_id               290611 non-null  int64         
 10  event_name             290611 non-null  object        
 11  event_type_main        290611 non-null  object        
 12  service_name           290611 non-null  obje

Датасет содержит 15 столбцов и 290611 строк, в которых представлена информация о пользователях и их заказах на сервисе `Я.Афиша`.

Описание данных:

- `user_id` — уникальный идентификатор пользователя, совершившего заказ;
- `device_type_canonical` — тип устройства, с которого был оформлен заказ ( mobile — мобильные устройства, desktop — стационарные);
- `order_id` — уникальный идентификатор заказа;
- `order_dt` — дата создания заказа (используйте данные created_dt_msk );
- `order_ts` — дата и время создания заказа (используйте данные created_ts_msk );
- `currency_code` — валюта оплаты;
- `revenue` — выручка от заказа;
- `tickets_count` — количество купленных билетов;
- `days_since_prev` — количество дней от предыдущей покупки пользователя, для пользователей с одной покупкой — значение пропущено;
- `event_id` — уникальный идентификатор мероприятия;
- `event_name` - название мероприятия в закодированном виде;
- `service_name` — название билетного оператора;
- `event_type_main` — основной тип мероприятия (театральная постановка, концерт и так далее);
- `region_name` — название региона, в котором прошло мероприятие;
- `city_name` — название города, в котором прошло мероприятие.

Наблюдения:
- имена столбцов имеют единый вид - snake case и не содержат пробелов
- данные в столбцах соответствуют описанию
- имеем хороший набор данных без пропусков (пропуски только в столбце `days_since_prev`, но мы сами их туда поставили и знаем о чём они говорят)
- по возможности можно уменьшить размер занимаемой памяти, поменяв данные на подходящий тип
- большинство данных имеет тип `object` - 8 столбцов, 2 столбца с данными `datetime64`, целочисленный тип `int64` - 3 столбца, `float64` - 2

In [8]:
# сделаем копию датасета

df_original = df.copy()

# проверим

len(df_original)

290611

## Предобработка данных

In [9]:
# посмотрим в какой валюте представленны данные о выручке

print(df['currency_code'].value_counts())

currency_code
rub    285542
kzt      5069
Name: count, dtype: int64


Основная часть данных - рубли, присутствуют тенге. Приведём выручку к рублю.

In [10]:
# загружаем датасет с курсом тенге к рублю

tenge_df = pd.read_csv('https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')

tenge_df.head()

,data,nominal,curs,cdx
0,2024-01-10,100,19.9391,kzt
1,2024-01-11,100,19.7255,kzt
2,2024-01-12,100,19.5839,kzt
3,2024-01-13,100,19.4501,kzt
4,2024-01-14,100,19.4501,kzt


In [11]:
tenge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   data     357 non-null    object 
 1   nominal  357 non-null    int64  
 2   curs     357 non-null    float64
 3   cdx      357 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 11.3+ KB


In [12]:
# преобразуем дату для присоединения

tenge_df['data'] = pd.to_datetime(tenge_df['data'])

print(f"Тип даты после преобразования: {tenge_df['data'].dtype}")
print()

# посчитаем курс для одного тенге

tenge_df['curs_for_one'] = tenge_df['curs'] / 100

tenge_df.head()

Тип даты после преобразования: datetime64[ns]



,data,nominal,curs,cdx,curs_for_one
0,2024-01-10,100,19.9391,kzt,0.199391
1,2024-01-11,100,19.7255,kzt,0.197255
2,2024-01-12,100,19.5839,kzt,0.195839
3,2024-01-13,100,19.4501,kzt,0.194501
4,2024-01-14,100,19.4501,kzt,0.194501


In [13]:
# переименовываем столбец 'data' в 'order_dt' для объединения

tenge_df = tenge_df.rename(columns={'data': 'order_dt'})

In [14]:
df = df.merge(tenge_df[['order_dt', 'curs_for_one']], on='order_dt',how='left')

df.head()

,user_id,device_type_canonical,order_id,order_dt,order_ts,currency_code,revenue,tickets_count,days_since_prev,event_id,event_name,event_type_main,service_name,region_name,city_name,curs_for_one
0,0002849b70a3ce2,mobile,4359165,2024-08-20,2024-08-20 16:08:03,rub,1521.94,4,NaN,169230,f0f7b271-04eb-4af6-bcb8-8f05cf46d6ad,театр,Край билетов,Каменевский регион,Глиногорск,0.186972
1,0005ca5e93f2cf4,mobile,7965605,2024-07-23,2024-07-23 18:36:24,rub,289.45,2,NaN,237325,40efeb04-81b7-4135-b41f-708ff00cc64c,выставки,Мой билет,Каменевский регион,Глиногорск,0.183419
2,0005ca5e93f2cf4,mobile,7292370,2024-10-06,2024-10-06 13:56:02,rub,1258.57,4,75.0,578454,01f3fb7b-ed07-4f94-b1d3-9a2e1ee5a8ca,другое,За билетом!,Каменевский регион,Глиногорск,0.196475
3,000898990054619,mobile,1139875,2024-07-13,2024-07-13 19:40:48,rub,8.49,2,NaN,387271,2f638715-8844-466c-b43f-378a627c419f,другое,Лови билет!,Североярская область,Озёрск,0.185010
4,000898990054619,mobile,972400,2024-10-04,2024-10-04 22:33:15,rub,1390.41,3,83.0,509453,10d805d3-9809-4d8a-834e-225b7d03f95d,стендап,Билеты без проблем,Озернинский край,Родниковецк,0.196648


In [15]:
df['curs_for_one'].isna().any()

np.False_

In [16]:
df['currency_code'].unique()

array(['rub', 'kzt'], dtype=object)

In [17]:
def convert_to_rub(row):
    if row['currency_code'] == 'kzt':
        return row['revenue'] * row['curs_for_one']
    else:
        return row['revenue']

df['revenue_rub'] = df.apply(convert_to_rub, axis=1)


In [18]:
df[df['currency_code'] == 'kzt'].head()


,user_id,device_type_canonical,order_id,order_dt,order_ts,currency_code,revenue,tickets_count,days_since_prev,event_id,event_name,event_type_main,service_name,region_name,city_name,curs_for_one,revenue_rub
70,0033403583a55ed,mobile,7850214,2024-09-17,2024-09-17 16:52:06,kzt,518.10,4,20.0,559919,f5ec2467-ee48-42f3-89a0-a2d90d1d6fce,другое,Билеты без проблем,Верхоречная область,Серебрянка,0.190125,98.503762
89,0040e12d70fae81,desktop,8634925,2024-09-02,2024-09-02 19:49:14,kzt,347.18,3,39.0,559476,61ec7c0e-ea0c-4a3e-a63a-09cbe1bc6177,другое,Билеты без проблем,Верхоречная область,Серебрянка,0.189330,65.731589
96,0054b38b2653e6e,mobile,7625522,2024-09-09,2024-09-09 18:06:04,kzt,328.77,4,NaN,559582,524f434e-0381-4f47-9688-7c4e41961b65,другое,Билеты без проблем,Верхоречная область,Серебрянка,0.185991,61.148261
277,00f4f5244501ce6,mobile,720419,2024-06-11,2024-06-11 20:15:57,kzt,22021.55,5,NaN,536826,76af4f88-c554-4e9e-9df4-1cc8aeec8959,концерты,Мой билет,Верхоречная область,Серебрянка,0.198928,4380.702898
460,01370a0e9c1a7d5,mobile,2229811,2024-06-04,2024-06-04 13:19:47,kzt,7397.66,4,1.0,533803,0686ac7b-6f3e-4177-bb73-16b54acd1223,концерты,Мой билет,Шанырский регион,Широковка,0.199833,1478.296591
